# 2024-12-06

## Data

In [1]:
arr = []
open("../inputs/2024-12-06.txt", "r") do file
    for line in eachline(file)
        chars = split(line, r"")
        push!(arr, String.(chars))
	end
end

n = length(arr)
m = length(arr[1])
mat = Matrix{String}(undef, n,m)

for i in 1:n
    for j in 1:m
        mat[i,j] = arr[i][j]
    end
end

## Part I

In [2]:
mat[ [initial_position[1], initial_position[2]] + [1,0] ]

UndefVarError: UndefVarError: `initial_position` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [61]:
function steps_till_leave(mat)
    pos = findfirst(x -> x in ["^", "v", "<", ">"], mat)
    dir = Dict(
        "^" => [-1, 0],
        "v" => [1, 0],
        "<" => [0, -1],
        ">" => [0, 1],
    )[mat[pos]]
    dir = CartesianIndex(dir...)
    R = [
        0 1;
        -1 0
    ]
    visited = []
    finished = false
    while !finished
        ray = [pos + j * dir for j in 1:n if checkbounds(Bool, mat, pos + j * dir)]
        if length(ray) == 0
            finished = true
            continue
        end
        k = findfirst(x -> x == "#", mat[ray])
        if k === nothing 
            finished = true
            mat[ray] .= "X"
            append!(visited, [(r, dir) for r in ray])
            continue
        end
        if k > 1
            pos = ray[k - 1]
        end

        dir = CartesianIndex((R * [dir[1], dir[2]])...)
        if (pos, dir) in visited
            return Inf
        end
        if k > 1
            append!(visited, [(r, dir) for r in ray[1:(k-1)]])
            mat[ray[1:(k-1)]] .= "X"
        else
            append!(visited, [(pos, dir)])
        end
    end
    return count(x -> x == "X", mat) 
end

steps_till_leave (generic function with 1 method)

In [27]:
steps_till_leave([
    "." "#" "#" ".";
    "#" "." "." "#";
    "#" "^" "." "#";
    "." "#" "#" "."
])

Inf

In [28]:
steps_till_leave(copy(mat))

4752

## Part II

In [62]:
mat2 = copy(mat)
pos = findfirst(x -> x in ["^", "v", "<", ">"], mat2)

dir = Dict(
    "^" => [-1, 0],
    "v" => [1, 0],
    "<" => [0, -1],
    ">" => [0, 1],
)[mat2[pos]]
dir = CartesianIndex(dir...)
R = [
    0 1;
    -1 0
]
visited = []
finished = false
while !finished
    ray = [pos + j * dir for j in 1:n if checkbounds(Bool, mat2, pos + j * dir)]
    k = findfirst(x -> x == "#", mat2[ray])
    if k === nothing
        finished = true
        mat2[ray] .= "X"
        continue
    end
    pos = ray[k - 1]
    dir = CartesianIndex((R * [dir[1], dir[2]])...)
    append!(visited, [(r, dir) for r in ray[1:(k-1)]])
    mat2[ray[1:(k-1)]] .= "X"
end

function results_in_cycle(pos, mat)
    mat2 = copy(mat)
    mat2[pos] = "#"
    
    return steps_till_leave(copy(mat2)) == Inf
end


results_in_cycle (generic function with 1 method)

In [ ]:
# I don't know what goes wrong at 3302
sum([
    results_in_cycle(ind, copy(mat)) for ind in findall(x -> x == "X", mat2)[[i for i in 1:length(findall(x -> x == "X", mat2)) if i!=3302]]
])

1719